# Huawei Research France

## Optical network modelling RAMP: cascading regressors

### Predict the output of a 32-channel optical network from its input and side information about the sequence of network components

_Balázs Kégl, Ludovic Dos Santos, Hartmut Hafermann (Huawei Research, Noah's Ark and Optical Laboratories, France)_

## Introduction

Building system models and simulators is a relatively new application area of machine learning. These models can replace expensive experimentation and accelerate design. This year's Huawei France hackathon falls into this category: we will ask participants to learn a model of optical networks from (real) data taken in our Optical Lab. It is a **32-output nonlinear regression problem**, with non-classical challenges:
1. **Transfer learning**: the training and test data will be different: at training we will provide data measured at internal points of the network, whereas we will test the models only for the full cascades, taken on a network with different characteristics from the training network.
2. **Non-standard input**: besides input-output data, we will also provide variable-length side information about the network components which may greatly help the transfer from the training network to the test network.
3. **Scoring metrics** will be a nonstandard measure of the relative prediction error that covers 99% test instances.

### Context

In optical transmission links, information is transmitted through optical fibers. The available transmission bandwidth is equally divided into a fixed number of slots or channels, 32 in this data challenge. Each channel can be used to transmit a beam of laser light at a fixed frequency, modulated by an information carrying signal, such that it occupies its assigned bandwidth. Each of these channels can be used to independently transmit information through the fiber.

Even though modern fibers are highly transparent, the **loss of signal strength** is not negligible and can be as high as a factor of 100 = 20[dB](https://en.wikipedia.org/wiki/Decibel) for 100km of fiber. Fiber transmission links therefore typically consist of a **cascade of alternating optical amplifiers and fibers**. Optical amplifiers compensate for the losses by amplifying the optical signal directly inside the fiber, without the need to convert it into an electrical signal. One of the most widely used type of amplifier is the [Erbium-doped fiber amplifier (EDFA)](https://en.wikipedia.org/wiki/Optical_amplifier#Erbium-doped_optical_fiber_amplifiers). The optical link considered here consists of EDFAs, [Single mode fibers (SMFs)](https://en.wikipedia.org/wiki/Single-mode_optical_fiber) and variable optical attenuators (VOAs). The dashed lines in the following figure indicate the positions at which the signal is measured.
<img src="https://raw.githubusercontent.com/ramp-kits/optical_network_modelling/2977eff1ac82349c2a130e89157e2e9db6a79d6a/link.png">

An EDFA implements a **32-input 32-output function**. Computer models of this function are highly useful when telecom providers design  optical networks since these models allow them to optimize the network before building it. In principle, EDFA amplifies its 32 input signals linearly, controlled by two parameters (set by the operator of the network), its nominal gain (the factor by which the signal is amplified on average) and nominal tilt (the factor by which the gain differs for leftmost and rightmost channel). In practice, the **amplification is nonlinear** and it also **depends on which channels are switched on and off**. Generalizing to unseen on/off configurations is important since network operators regularly add or drop channels according to transmission demand. There is no simple physical model of this combinatorial function, thus the aim of our project: explore using machine learning techniques that can be learned on a small subsample of the combinatorial space.

Similar to an EDFA, a fiber is a nonlinear 32-input 32-output function whose response depends on which channels are switched on and off. The VOAs reduce the power of each channel by the same factor. They simply model the fact that in real optical links losses can occur, for example through poor splices (where two fibers are fused together). Here for simplicty we treat the composition of fiber and VOA as one entity. 

### The data

The data is taken from lab measurements of the channel power in the cascaded link shown in the figure above. For the purpose of this RAMP, we can abstract the transmission cascade as a concatenation of eight modules:
<img src="https://raw.githubusercontent.com/ramp-kits/optical_network_modelling/2977eff1ac82349c2a130e89157e2e9db6a79d6a/cascade.png">
Each module is either an EDFA or an SMF (fiber+VOA), described by a small metadata vector whose format depends on the module type. Each **training instance will be a subcascade** $(i,j)$ with input signal $P_i \in \mathbb{R}^{32}$ and output signal $P_j \in \mathbb{R}^{32}$ and a sequence of module metadata $(m_{i+1}, \ldots, m_j)$. Each **test instance will be a full cascade** $(i=0,j=8)$ with input signal $P_0 \in \mathbb{R}^{32}$ and output signal $P_8 \in \mathbb{R}^{32}$ and a sequence of module metadata $(m_1, \ldots, m_8)$. 

Each component of the vectors $P_j$ represents the power of the corresponding channel. Since the measured power at the entry and exit of each module differs by large factors, each channel was normalized to its power measured in the configuration where all channels are on. Powers are given on linear scale (not in dB). The values are therefore close to unity.

For each module, the metadata is a vector with two values. For an EDFA, these are the nominal gain and nominal tilt, $m_i=[G_i, T_i]$ for $i$ odd, both measured in dB. The fibers in this dataset are of the same type and length. For the SMF module, the metadata are the factors by which the channels are attenuated in the VOA before or after the fiber, measured in dB: $m_i=[\alpha_i^\text{in}, \alpha_i^\text{out}]$ ($i$ even). The absence of a VOA corresponds to 0 dB. The channel power at the entrance of the fiber can in principle be calculated by dividing the power at the entrance of the module by $10^{\alpha^\text{in}/10}$. The power at the exit of the fiber is correspondingly obtained by multiplying by $10^{\alpha^\text{out}/10}$.

To mimic real-life scenarios, the module metadata of the test cascade will be different from the 
module metadata of the training cascade (although the module types in the sequence will be the same). Each training and test instance will be generated by a random channel on/off configuration, cascaded through the network (so subcascade instances will not be independent). Again, training and test configurations will be different.

### The scoring metrics

We will report a classical RMSE score on the submissions, but the official ranking score will be EM99, which represents an error margin (the uncertainty in the prediction) within which 99% of the test cases fall. Specifically, it is given by the 99th percentile of the relative errors of the test predictions, measured in dB. All the metrics are only computed over channels which are switched on (non-zero components of the $P_i$).


## Competition rules

The rules may be adjusted before the start of the challenge, November 4 2020.

[//]: # "* Submissions will be trained on a time series of roughly 5000 time steps and tested on a time series of roughly 20000 time steps." 
* The competition will end on November 13, 2020 at 18h UTC (20h in Paris).
* The competitive phase will be followed by a collaborative phase when participants may (and encouraged to) reuse and combine each other's code.
* The final results will be announced at the closing ceremony on November 18. We will also ask top teams to present their approaches at the event.
* All models will be trained on the same cloud server allowing 4 CPUs (with shared memory of 128GB RAM).
* Participants will be given a total of 20 machine hours (per cross-validation fold). Submissions of a given participant will be ordered by submission timestamp. We will make an attempt to train all submissions, but starting from (and including) the first submission that makes the participant's total training time exceed 20 hours, all submissions will be disqualified from the competition (but can enter into the collaborative phase). Testing time will not count towards the limit. Training time will be displayed on the leaderboard for all submissions, rounded to second. If a submission raises an exception, its training time will not count towards the total.
* There is a timeout of 1 day between submissions that did not raise an exception.
* Submissions submitted after the end of the competition will not qualify for prizes.
* The public leaderboard will display validation scores running a cross-validation. The official scores will be calculated on the hidden test set and will be published after the closing of the competition. We will rank submissions according to their EM99 score.
* The organizers will do their best so that the provided backend runs flawlessly. We will communicate with participants in case of concerns and will try to resolve all issues, but we reserve the right to make unilateral decisions in specific cases, not covered by this set of minimal rules.
* The organizers reserve the right to disqualify any participant found to violate the fair competitive spirit of the challenge. Possible reasons, without being exhaustive, are multiple accounts, attempts to access the test data, etc.
* Participants can form teams outside the platform before submitting any model individually, and submit on a single team account. Participating in more than one team at the same time is against the "no multiple accounts" rule, so, if discovered, may lead to disqualification. Before signing up, teams should communicate their composition and team name to BeMyApp.
* Participants retain copyright on their submitted code and grant reuse under BSD 3-Clause License.

Participants accept these rules automatically when making a submission at the RAMP site.

## Getting started

Besides the usual pydata libraries, you will need to install `ramp-workflow` from the advanced branch:
```
pip install git+https://github.com/paris-saclay-cds/ramp-workflow.git@advanced
```


It will install the `rampwf` library and the `ramp-test` script that you can use to check your submission before submitting. You do not need to know this package for participating in the challenge, but it could be useful to take a look at the [documentation](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/advanced/index.html) if you would like to know what happens when we test your model, especially the [RAMP execution](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/advanced/scoring.html) page to understand `ramp-test`, and the [commands](https://paris-saclay-cds.github.io/ramp-docs/ramp-workflow/advanced/command_line.html) to understand the different command line options. 

In [1]:
import numpy as np
import rampwf as rw

Read `problem.py` so you can have an access to the same interface as the testing script.

In [8]:
problem = rw.utils.assert_read_problem()

### The data

First take the public data set from the Slack team #optical_network_challenge channel (join by [clicking here](https://join.slack.com/t/huaweiramp/shared_invite/zt-i5hqij3l-ufzyUJgKNJA407sWNB1QDA)) and unzip it to create `data/c1..c4`. 

Read the training and test data.

In [9]:
X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

The input data is a list of thruples of metadata, input power, and campaign index. The metadata is a sequence of desciptors of the network elements of the subcascade. 

In [10]:
X_train[1]

array([list([('EDFA', [24.0, 20.0]), ('SMF', [5.6, 0])]),
       list([0.9854011268786588, 0.9579909333275516, 0.9874405077601779, 0.966566146485512, 0.9784920440057496, 0.9726006367972836, 0.9843132422152838, 0.9784334877724056, 0.9709820621986954, 0.9820604922371035, 0.9826946765531455, 0.9773055594854204, 0.968574860637684, 0.9762821604505046, 0.975065053383652, 0.975329686757004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
       1], dtype=object)

The training data contains all subcascades of campaigns 1 and 2, plus some full cascades from campaigns 3 and 4. The test data contains the rest of the full cascades from campaigns 3 and 4. The reason for this setup is the following. In real life when a network is built, we will be able to have end-to-end measurements on the full network but not on its subcascades. In addition, we will have a large number of laboratory measurements on various subcascades, but the components of these lab networks will not have the same metadata as the components of the real network. In the challenge, the training campaigns represent the lab measurements and the test campaigns represent the end-to-end field measurements.

In [11]:
print('training campaigns = {}'.format(np.unique([x[2] for x in X_train])))
print('test campaigns = {}'.format(np.unique([x[2] for x in X_test])))
print('training sequence lengths = {}'.format(np.unique([len(x[0]) for x in X_train])))
print('test sequence lengths = {}'.format(np.unique([len(x[0]) for x in X_test])))
print('training sequence lengths from test campaigns = {}'.format(
    np.unique([len(x[0]) for x in X_train if x[2] in problem._test_campaigns])))

training campaigns = [1 2 3 4]
test campaigns = [3 4]
training sequence lengths = [1 2 3 4 5 6 7 8]
test sequence lengths = [8]
training sequence lengths from test campaigns = [8]


Since the test only contains full cascades, the training set is much larger than the test set, but it contains only a very small number of instances that come from the same distribution as the test set. Also note that the training subcascades also come from a small number of full cascades of the training campaigns; the information in these subcascades is redundant.

In [12]:
print('training size = {}'.format(len(X_train)))
print('test size = {}'.format(len(X_test)))
print('number of training full cascades from the same distribution '
      'as the test set = {}'.format(len([x for x in X_train if x[2] in problem._test_campaigns])))
print('number of full cascades from the training campaigns = {}'.format(
    len([x for x in X_train if x[2] in problem._train_campaigns and len(x[0]) == 8])))

training size = 23055
test size = 1213
number of training full cascades from the same distribution as the test set = 303
number of full cascades from the training campaigns = 632


The second element of each instance is the actual input power to the subcascades, an array of 32 real numbers. The target corresponds to the output of the subcascade on the same channels. In each instance, we turned on about a third of the channels, the rest are zeros. The actual values in the arrays are factors wrt a mean power so the overall mean of input and output values is about 1. We will measure the performance of the predictors only on channels that were turned on. The overall standard deviation of the nonzero outputs is an upper bound of the RMSE of a predictor. Returning the input as the output is also a good baseline: it is slightly better than returning the mean on the training set (dominated by shorter subcascades) but not on the test set that contains only long full cascades.

In [16]:
channel_input_train = np.array([x[1] for x in X_train])
n_train = len(channel_input_train)
mask_nonzero_train = channel_input_train > 0
print('Mean occupation of channels (power != 0): {}'.format(
    np.sum(mask_nonzero_train) / (problem._NB_CHANNELS * n_train)))
print('Mean input power factor of nonzero channels = {}'.format(
    channel_input_train[mask_nonzero_train].mean()))
print('Std input power factor of nonzero channels = {}'.format(
    channel_input_train[mask_nonzero_train].std()))
print('Mean output power factor of nonzero channels = {}'.format(
    y_train[mask_nonzero_train].mean()))
print('Std output power factor of nonzero channels = {}'.format(
    y_train[mask_nonzero_train].std()))
print('RMSE of the identity function on nonzero channels = {}'.format(
    np.sqrt(np.mean((channel_input_train[mask_nonzero_train] -
                     y_train[mask_nonzero_train]) ** 2))))

Mean occupation of channels (power != 0): 0.323546952938625
Mean input power factor of nonzero channels = 1.0167000697991586
Std input power factor of nonzero channels = 0.0647956525418062
Mean output power factor of nonzero channels = 1.0452292762288091
Std output power factor of nonzero channels = 0.1288248003299997
RMSE of the identity function on nonzero channels = 0.10789475221344981


In [17]:
channel_input_test = np.array([x[1] for x in X_test])
n_test = len(channel_input_test)
mask_nonzero_test = channel_input_test > 0
print('Mean occupation of channels (power != 0): {}'.format(
    np.sum(mask_nonzero_test) / (problem._NB_CHANNELS * n_test)))
print('Mean input power factor of nonzero channels = {}'.format(
    channel_input_test[mask_nonzero_test].mean()))
print('Std input power factor of nonzero channels = {}'.format(
    channel_input_test[mask_nonzero_test].std()))
print('Mean output power factor of nonzero channels = {}'.format(
    y_test[mask_nonzero_test].mean()))
print('Std output power factor of nonzero channels = {}'.format(
    y_test[mask_nonzero_test].std()))
print('RMSE of the identity function on nonzero channels = {}'.format(
    np.sqrt(np.mean((channel_input_test[mask_nonzero_test] -
                     y_test[mask_nonzero_test]) ** 2))))

Mean occupation of channels (power != 0): 0.3230626545754328
Mean input power factor of nonzero channels = 1.0122318368841638
Std input power factor of nonzero channels = 0.042492824657587844
Mean output power factor of nonzero channels = 1.029272143408915
Std output power factor of nonzero channels = 0.15197441257292613
RMSE of the identity function on nonzero channels = 0.16637156577695347


### The prediction task

You can load here the regressor implemented in the starting kit. It is a simple linear regressor that ignores the metadata.

In [27]:
# %load submissions/starting_kit/regressor.py
import numpy as np
from sklearn.linear_model import Ridge


class Regressor:
    def __init__(self):
        pass

    def fit(self, X, y):
        X_array = np.array([np.array(X_i) for X_i in X[:, 1]])
        self.reg = Ridge(alpha=1)
        self.reg.fit(X_array, y)

    def predict(self, X):
        X_array = np.array([np.array(X_i) for X_i in X[:, 1]])
        return self.reg.predict(X_array)


You can train your submission and obtain test predictions using the same protocol as our training script.

In [28]:
trained_workflow = problem.workflow.train_submission('submissions/starting_kit', X_train, y_train)
y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)

### The scores

We compute three scores on the predictions. All scores are implemented in `problem.py` so you can look at the precise definitions there. All scores take into consideration only channels that are "on" in the particular instance. **All scores check the positivity of all predictions and return $\infty$ if any of the predictions are negative.**

EM99 is the 99 percentile of the relative error measured in decibel. MEM is the worst relative error measured in decibel. RMSE is the classical root mean squared error (except that it is also only computed on "on" channels).

**The official score of the competition is EM99.**

In [34]:
em99_score = problem.score_types[0]
rmse_score = problem.score_types[1]
mem_score = problem.score_types[2]

In [35]:
print('EM99 test score = {}'.format(em99_score(y_test, y_test_pred)))
print('RMSE test score = {}'.format(rmse_score(y_test, y_test_pred)))
print('MEM test score = {}'.format(mem_score(y_test, y_test_pred)))

EM99 test score = 1.2808269708777709
RMSE test score = 0.12475812286907513
MEM test score = 2.376883475779602


### The cross validation scheme

The cross-validation follows the same scheme as the train/test cut (see `problem.get_cv`): only (part) of the test campaign full cascades are in the validation sets and the rest of the test campaign instances plus a part of the training subcascades are in the training sets. You are free to play with both the train/test cut and the cross-validation when developing your models but be aware that we will use the same scheme on the official server as the one in the RAMP kit (on a different set of four campaigns that will not be available to you).

The following cell goes through the same steps as the official evaluation script (`ramp-test`).

In [36]:
splits = problem.get_cv(X_train, y_train)
y_test_preds = []
for fold_i, (train_is, valid_is) in enumerate(splits):
    trained_workflow = problem.workflow.train_submission(
        'submissions/starting_kit', X_train, y_train, train_is)
    y_train_pred = problem.workflow.test_submission(trained_workflow, X_train[train_is])
    y_valid_pred = problem.workflow.test_submission(trained_workflow, X_train[valid_is])
    y_test_pred = problem.workflow.test_submission(trained_workflow, X_test)
    print('training EM99 on fold {} = {}'.format(
        fold_i, em99_score(y_train[train_is], y_train_pred)))
    print('validation EM99 on fold {} = {}'.format(
        fold_i, em99_score(y_train[valid_is], y_valid_pred)))
    print('test EM99 on fold {} = {}'.format(
        fold_i, em99_score(y_test, y_test_pred)))
    y_test_preds.append(y_test_pred)

training EM99 on fold 0 = 1.096709749334698
validation EM99 on fold 0 = 1.2291169010842582
test EM99 on fold 0 = 1.2843604038060734
training EM99 on fold 1 = 1.1078058921349596
validation EM99 on fold 1 = 1.2841345341739479
test EM99 on fold 1 = 1.2764197042868608
training EM99 on fold 2 = 1.0913384768485284
validation EM99 on fold 2 = 1.2792855072522518
test EM99 on fold 2 = 1.280730271669307
training EM99 on fold 3 = 1.0989618938773276
validation EM99 on fold 3 = 1.2151060503006859
test EM99 on fold 3 = 1.2912051839696272
training EM99 on fold 4 = 1.1307055522166911
validation EM99 on fold 4 = 1.2585370688329733
test EM99 on fold 4 = 1.2781598166718113
training EM99 on fold 5 = 1.0997655190735045
validation EM99 on fold 5 = 1.2262019614883615
test EM99 on fold 5 = 1.2857076289178684
training EM99 on fold 6 = 1.1228302433652817
validation EM99 on fold 6 = 1.2739918097896585
test EM99 on fold 6 = 1.2789111456373359
training EM99 on fold 7 = 1.1058643401979855
validation EM99 on fold 7 

We compute both the mean test score and the score of bagging your eight models. The official ranking will be detemined by the bagged test score (on different data sets from the ones you have). Your public score will be the bagged validation score (the averaging is [slightly more complicated](https://github.com/paris-saclay-cds/ramp-workflow/blob/master/rampwf/utils/combine.py#L56) since we need to take care of the cross validation masks properly). 

In [23]:
bagged_y_pred = np.array(y_test_preds).mean(axis=0)
print('Mean EM99 score = {}'.format(
    np.mean([em99_score(y_test, y_test_pred) for y_test_pred in y_test_preds])))
print('Bagged EM99 score = {}'.format(
    em99_score(y_test, np.array(y_test_preds).mean(axis=0))))

Mean EM99 score = 0.12482309289565047
Bagged EM99 score = 0.12480803802205302


## Example submissions

Besides the starting kit implementing a linear regressor and the identity 
regressor, we provide you simple neural net based submissions in PyTorch, 
to get you started.
Even if they are implemented in PyTorch and that you choose to go with an other
library (TensorFlow, XGBoost, scikit-learn, ...)
going through those examples might help. For those not familiar with it but 
willing to use this library consider this 
[tutorial](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html).

Note that those submissions are not fully hyperparameters-optimized and do not
come with every possible Deep Learning tricks. Feel free to play with it
locally.
There is no need to submit them since their respective score are already in the
leaderboard.
In the following we will look at each of them to see some important points,
for data format explanations and statistics please go to the data section.
The following cells are made from some non executable 
code extract of the submissions to emphasis on important parts.

### A simple Multi-Layer Perceptron

This model is submitted under `submission/single_mlp/regressor.py`.
This model take as input the input power ```p_in``` of the cascade and outputs
the estimated output power without considering any meta data
such as the cascade length or the feature of the optical modules. 

We define your Regressor as a PyTorch model defining the modules in the
```__init__()``` function. It consists of two fully connected layers with an
hidden size of 128. It takes as input the 32 channel input power vector and outputs a
32 channel output power prediction. 

In [ ]:
def __init__(self):
    super(Regressor, self).__init__()
    # Definition of the modules of the model
    # Two fully connected layers
    self.fc0 = torch.nn.Linear(32, 128)
    self.fc1 = torch.nn.Linear(128, 32)
 

We then implement its corresponding forward function adding a ```tanh```
activation function.

In [ ]:
def forward(self, p_in):
    # Compute the output of the model using a tanh activation function
    p_out = self.fc1(torch.tanh(self.fc0(p_in)))
    # Return positive values when evaluating the model
    return p_out if self.training else relu(p_out)

Note that when the model is in evaluation mode (calling self.train() sets
```self.training``` to ```True```, and calling ```self.eval()``` sets it to
```False```) we clip the values to be positive. Outputting some negative values
when predicting will end up with ```inf``` scores. This clipping should be
done even if using other library.

For those familiar with PyTorch, the ```fit``` RAMP function corresponds to a
somehow classic learning scheme.  

In [ ]:
def fit(self, X, y):
    # Turn on training mode
    self.train()
    # Get data and create train data loaders
    data_as_list = [
        [torch.tensor(p_in).float(), torch.tensor(p_out).float()]
        for (_, p_in, _), p_out in zip(X, y)]
    train_loader = torch.utils.data.DataLoader(data_as_list,
                                               batch_size=128)
    # Instantiate criterion and optimizer
    crit = torch.nn.MSELoss()
    opt = torch.optim.Adam(self.parameters())

    # Training loop
    for e in range(100):
        for p_in, p_out in train_loader:
            opt.zero_grad()
            preds = self(p_in)
            # Since the evaluation is only done for on-channels it
            # helps the optimization to only backpropagate through them.
            on_chan = p_in != 0
            on_preds = torch.mul(on_chan, preds)
            on_p_out = torch.mul(on_chan, p_out)
            loss = crit(on_preds, on_p_out)
            loss.backward()
            opt.step()

We format the data to feed easily a PyTorch ```DataLoader``` in order to
iterate among the training examples. Then we define our criterion
(here an ```MSELoss```) and our optimizer (here ```Adam```).

Note that for this model, since the RAMP scores consider only channels that are
on, backpropagating only through on channels seems to bring better results.

Finally, the RAMP ````predict``` function is just a forward pass of the model.

In [ ]:
def predict(self, X):
    # Turn on evaluation mode
    self.eval()
    # No ground truth when predicting
    p_in = torch.stack([torch.tensor(p_in).float() for _, p_in, _ in X])
    preds = self(p_in).detach().numpy()
    return preds

### Cascading MLPs

The second example we provide should help you cascade module wise models
and use some of the meta data included in ```X``` (all but the campaign
identifier).
The model we built consist in learning a specific MLP for each module present
in the training set (there is no unseen modules in the test set) that take as
input ```p_in``` and the features of the module and output some predictions.

To ease the comprehension of the code and making it easily reusable to you we
define the PyTorch model outside the RAMP Regressor class, but we could have
defined the model with hard coded parameters as we did for the previous MLP. 

This model is submitted under `submission/cascade_mlp/regressor.py`.


First let's define our PyTorch model that consists of a list of MLPs, one
for each optical module type. The optical module information is listed into
```mod_info``` that is defined in the RAMP ```fit``` function.

In [ ]:
class PyTorchModel(torch.nn.Module):
    def __init__(self, mod_info):
        super(PyTorchModel, self).__init__()
        self.mod_info = mod_info
        # Construct as many MLPs as modules present in the data
        self.MLPs = torch.nn.ModuleList(
            [MLP(m["nb_feat"]) for m in self.mod_info])

During the forward of this model we need to check the cascade's length and the
optical modules present in order to apply (or not) the corresponding MLP.
We compute a mask ```msk``` that corresponds to cascades for which optical
modules are left and the current one having the same identifier as the current
MLP. We then update the value of ```p_out``` and return it when we end up 
predicting all cascades output power.

In [ ]:
def forward(self, mod_id_seq, mod_feat_seq, p_in):
    seq_len = torch.tensor(list(map(len, mod_feat_seq)))
    p_out = p_in
    max_nb_mod = max(seq_len)
    for n in range(max_nb_mod):
        for i, mlp in enumerate(self.MLPs):
            msk = torch.mul(mod_id_seq[:, n] == i, seq_len > n)
            if msk.any():
                feats = torch.stack(
                    [f[n] for i, f in enumerate(mod_feat_seq) if msk[i]])
                p_out[msk] = mlp(torch.cat([p_out[msk], feats], dim=-1))
    # Return positive values when evaluating the model
    return p_out if self.training else relu(p_out)

Since we have defined the model outside the Regressor class, the instantiation
of the model is done while calling the RAMP ```fit``` function. We first read
the data to retrieve some important information ```mod_info``` and then
instantiate our model.

In [ ]:
def fit(self, X, y):
    # Retrieve some information about the modules from the data
    all_mods = set(
        [(("type", mod[0]), ("nb_feat", len(mod[1]))) for seq, _, _ in X
         for mod in seq])
    mod_info = [dict(m) for m in all_mods]
    self.mod_id = {mod["type"]: i for i, mod in enumerate(mod_info)}

    # Instantiate the PyTorch model
    self.model = self.Model(mod_info)
    ...

The remaining difference between this fit and the previous one is the
formatting of the data we feed the ```DataLoader``` with that now includes meta
information about the module.

In [ ]:
data_list = [{"mod_id_seq": torch.tensor(
    [self.mod_id[mod] for mod, _ in mod_seq]),
    "mod_feat_seq_list": [torch.tensor(feat).float() for
                          _, feat in mod_seq],
    "input_power": torch.tensor(p_in).float(),
    "output_power": torch.tensor(p_out).float()} for
    (mod_seq, p_in, campaign_id), p_out in zip(X, y)]

Same changes are made in the RAMP ```predict``` function.

Since the data format couldn't be collated easily by the ```DataLoader```
(due to ```"mod_id_seq"``` having different possible lengths), we
define a specific one which stack every components of the batch.

In [ ]:
def collate_fn(batch):
    # Power output
    p_out = torch.stack([sample["output_power"] for sample in batch])
    # Power input
    p_in = torch.stack([sample["input_power"] for sample in batch])
    # Module id
    l_id_seq = [sample["mod_id_seq"] for sample in batch]
    mod_id_seq = pad_sequence(l_id_seq, batch_first=True, padding_value=-1)
    # Module features
    mod_feat_seq = [sample["mod_feat_seq_list"] for sample in batch]

    return (mod_id_seq, mod_feat_seq, p_in), p_out

### Learning cnosidering only single optical module data 

The third example we provide should help you implement models where you want to
have different behaviours while learning based on meta data. Here we propose a
model where the learning is done using only single module data to see how
well it could generalize to cascade. Same scheme could be use if you want to
have different behaviors based on the cascade identifier for example.

This model is submitted under `submission/cascade_mlp_module/regressor.py`.

The main differences from the previous example are in the forward function of
the model and an exception in the backward. In this particular example
filtering the data from the beginning would have fix the backward issue. But
you may find some cases where checking the loss properties will be mandatory,
in this case do a backward only when it makes sense.

In [ ]:
if loss.requires_grad:
    loss.backward()

As for returning postive values at evaluation time, the use of PyTorch
```torch.nn.Module``` argument ```training``` ease the distinction between
train mode (calling ```self.train()``` in the ```fit``` function) and eval mode
(calling ```self.eval()``` in the predict one) of the model. In this example,
we want to only backpropagate through cascade of length 1
(i.e. ```seq_len == 1```).

In [ ]:
def forward(self, mod_id_seq, mod_feat_seq, p_in):
    seq_len = torch.tensor(list(map(len, mod_feat_seq)))
    p_out = p_in
    if self.training:
        # Training done on single modules
        # returning p_in for cascade (no backpropagation)
        for i, m in enumerate(self.MLPs):
            msk = torch.mul(mod_id_seq[:, 0] == i, seq_len == 1)
            if msk.any():
                feats = torch.stack(
                    [f[0] for i, f in enumerate(mod_feat_seq) if msk[i]])
                p_out[msk] = m(torch.cat([p_out[msk], feats], dim=-1))
        return p_out

    else:
        # Concatenate MLP to evaluate cascades
        max_nb_mod = max(seq_len)
        for n in range(max_nb_mod):
            for i, m in enumerate(self.MLPs):
                msk = torch.mul(mod_id_seq[:, n] == i, seq_len > n)
                if msk.any():
                    feats = torch.stack(
                        [f[n] for i, f in enumerate(mod_feat_seq) if
                         msk[i]])
                    p_out[msk] = m(torch.cat([p_out[msk], feats], dim=-1))
        return relu(p_out)

Once again, do not hesitate to play with those examples locally to get familiar
with RAMP and the data.

## Local testing (before submission)

You submission will contain a single `regressor.py` file implementing a regressor class with a `fit` and `predict` function (scikit-learn API) as in the starting kit. You should place it in the `submission/<submission_name>` folder in your RAMP kit folder. To test your submission, go to your RAMP kit folder in a terminal and type
```
ramp-test --submission <submission_name>
```
It will train and test your submission much like we did it above in this notebook, and print the foldwise and summary scores. You can try it also in this notebook:

In [37]:
!ramp-test --submission starting_kit

Testing Optical network modelling
Reading train and test files from ./data/ ...
Reading cv ...
Training submissions\starting_kit ...
CV fold 0
	score   EM99    RMSE   MEM      time
	train  1.097  0.0808  4.30  0.277946
	valid  1.229  0.1199  2.10  0.261952
	test   1.284  0.1250  2.39  0.015651
CV fold 1
	score   EM99    RMSE   MEM      time
	train  1.108  0.0834  3.90  0.102995
	valid  1.284  0.1265  1.72  0.196524
	test   1.276  0.1245  2.40  0.010556
CV fold 2
	score   EM99    RMSE   MEM      time
	train  1.091  0.0818  3.62  0.116234
	valid  1.279  0.1277  2.10  0.189690
	test   1.281  0.1249  2.38  0.011603
CV fold 3
	score   EM99    RMSE   MEM      time
	train  1.099  0.0816  3.94  0.199146
	valid  1.215  0.1247  1.73  0.259609
	test   1.291  0.1252  2.38  0.016765
CV fold 4
	score   EM99    RMSE   MEM      time
	train  1.131  0.0830  3.89  0.121125
	valid  1.259  0.1170  2.10  0.247806
	test   1.278  0.1244  2.37  0.013509
CV fold 5
	score   EM99    RMSE   MEM      time
	train  1

## Submission

1. First you will need to sign up at the [RAMP site](https://ecs-90-84-188-77.compute.prod-cloud-ocb.orange-business.com). Your team will be approved shortly by a system admin who will check that you communicated your team nick and composition to BeMyApp.
2. You will then need a second sign-up, this time for the [optical network challenge](https://ecs-90-84-188-77.compute.prod-cloud-ocb.orange-business.com/events/optical_network_modelling_hackaton). If your site sign up was approved in the previous point, you should see a "Join event" button on the right of the top menu. This request will also be approved by a site admin.
3. Once you are signed up, you can start submitting (once a day). If you are happy with your local scores, copy-paste your submission at the [sandbox](https://ecs-90-84-188-77.compute.prod-cloud-ocb.orange-business.com/events/optical_network_modelling_hackaton/sandbox), press "submit now", name your submission, then give credits to which other submission you used (in the collaborative phase you will see only your own submissions in the list.
4. Your submission will be sent to train. It will either come back with an error or will be scored. You can follow the status at [my submissions](https://ecs-90-84-188-77.compute.prod-cloud-ocb.orange-business.com/events/optical_network_modelling_hackaton/my_submissions).
5. If there is an error, click on the error to see the trace. You can resubmit a failed submission **under the same name**, this will not count in your daily quota.
6. There is no way to delete trained submissions. In exceptional cases we can stop a submission that hasn't been scored yet so you can resubmit. We strongly suggest to finish training at least one fold locally (using `ramp-test`) before submitting so you can estimate the training time.
7. You can follow the scores of the other participants at the [public leaderboard](https://ecs-90-84-188-77.compute.prod-cloud-ocb.orange-business.com/events/optical_network_modelling_hackaton/leaderboard).
8. The public [competition leaderboard](https://ecs-90-84-188-77.compute.prod-cloud-ocb.orange-business.com/events/optical_network_modelling_hackaton/competition_leaderboard) displays the top submission (according to the public score) of each participant. You can change which of your submission enters the competition by pulling out the top submission. Click on the particular submission at [my submissions](https://ecs-90-84-188-77.compute.prod-cloud-ocb.orange-business.com/events/optical_network_modelling_hackaton/my_submissions) and click on the yellow button. The operation is reversible as many times you want, even after the competition deadline.

## Contact

You can contact the organizers in the Slack of the challenge, join by [clicking here](https://join.slack.com/t/huaweiramp/shared_invite/zt-i5hqij3l-ufzyUJgKNJA407sWNB1QDA). 